In [1]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import numpy as np
import pandas as pd
import textwrap
from pprint import pprint
import os
import re
import data_processing
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\arthu\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\arthu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import importlib
importlib.reload(data_processing)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\arthu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


<module 'data_processing' from 'c:\\Users\\arthu\\Documents\\GitHub\\ner-using-bert\\BERT_Experiment\\data_processing.py'>

In [8]:
path_to_test = "acerpi_dataset/test/raw_documents/"
path_to_train = "acerpi_dataset/train/raw_documents/"

#### Open Unannotated Documents
We get the unnanotatted texts and store each document as a key, value pair in a dictionary

In [9]:
documents = {}
# Get test documents that require manual labeling
#ALREADY LABELED
# for filename in os.listdir(path_to_test):
#    if filename.endswith(".txt"):
#       with open(os.path.join(path_to_test, filename), 'r', encoding="utf8") as f:
#          key = int(filename.replace('.txt', ''))
#          value = f.read()
#          documents[key] = value

# Get train documents that require manual labeling
for filename in os.listdir(path_to_train):
   if filename.endswith(".txt"):
      with open(os.path.join(path_to_train, filename), 'r', encoding="utf8") as f:
         key = int(filename.replace('.txt', ''))
         value = f.read()
         documents[key] = value

Next we store the documents in a DataFrame

In [15]:
df_docs = pd.DataFrame.from_dict(documents, orient='index')
df_docs.columns = ['text']
print(df_docs.shape)
df_docs.head()

(538, 1)


,text
25644,Documento gerado sob autenticação Nº AHX.976.7...
25645,Documento gerado sob autenticação Nº SDG.667.3...
25646,Documento gerado sob autenticação Nº ASL.670.4...
25647,Documento gerado sob autenticação Nº PUG.672.5...
25648,Documento gerado sob autenticação Nº ORY.674.6...


#### Data Cleaning
We will use the **data_processing.clear_text** function which was built to clean the specific document we are using here. Since we already know they will be used for NER, the objective of the cleaning is getting the text into a format which will be easy to split into sentences. Each sentence will later be used as an input to our neural network.
There are three aspects we are cleaning:
1. Removing line breaks as they make it very hard for the sentence tokenizer to correctly recognize the begining and ending of sentences in the text
2. Removing repeated whitespaces which are used for the visual formatting of the documents but will generate unnecessary tokens for our neural network.
3. There are a lot of law and document numbers in this documents. There isn't a consistent writing of these numbers which can start as "nº.123", "nº. 123" and "nº 123". We will padronize this occurences to appear as "nº 123" since the punctuation after "nº" makes it harder for the sentence tokenizer to correctly separate the sentences.

In [7]:
df_docs.loc[55505]['text']

'Documento gerado sob autenticação Nº RRM.458.680.FGN, disponível no\nendereço http://www.ufrgs.br/autenticacao\nDocumento certificado eletronicamente, conforme Portaria nº\n3362/2016, que institui o Sistema de Documentos Eletrônicos da UFRGS.\n1/1\nPORTARIA Nº             4534                  de  22/06/2018\nO PRÓ-REITOR DE PLANEJAMENTO E ADMINISTRAÇÃO DA UNIVERSIDADE FEDERAL DO RIO GRANDE\nDO SUL, no uso de suas atribuições, considerando o disposto na Portaria nº 7611, de 29 de setembro de\n2016,  e  tendo  em  vista  o  que  consta  dos  Processos  Administrativos  n°  23078.510719/2016-21  e\n23078.524008/2017-15, do Contrato nº 044/2017, da Lei 10.520/02 e ainda da Lei 8.666/93,\nRESOLVE:\n \nAplicar à Empresa NEVES E ROMANOSKLI LTDA - ME, CNPJ n.º 15.717.915/0001-90, conforme atestado pela\nfiscalização no doc. SEI nº 0817113 e 0925939, bem como pelo NUDECON (doc. SEI nº 1055527) no Processo\n23078.524008/2017-15, as seguintes sanções administrativas:\n \n1. ADVERTÊNCIA, previst

In [16]:
print(df_docs['text'].loc[55698])
df_docs['text'] = df_docs['text'].apply(data_processing.clear_text)
print(df_docs['text'].loc[55698])

Documento gerado sob autenticação Nº FIH.807.978.DS7, disponível no endereço
http://www.ufrgs.br/autenticacao
Documento certificado eletronicamente, conforme Portaria nº
3362/2016, que institui o Sistema de Documentos Eletrônicos da UFRGS.
1/1
PORTARIA Nº             4562                  de  25/06/2018
O PRÓ-REITOR DE GESTÃO DE PESSOAS DA UNIVERSIDADE FEDERAL DO RIO GRANDE DO SUL, no
uso de suas atribuições que lhe foram conferidas pela Portaria nº.7684, de 03 de outubro de 2016, do
Magnífico Reitor, e conforme o Laudo Médico n°49995,
RESOLVE
Designar, temporariamente, nos termos da Lei nº. 8.112, de 11 de dezembro de 1990, com redação
dada  pela  Lei  nº.9.527,  de  10  de  dezembro  de  1997,  a  ocupante  do  cargo  de  ASSISTENTE  EM
ADMINISTRAÇÃO, do Quadro de Pessoal desta Universidade, LUCIANA DENISE FLORES (Siape: 2081417 ),
 para substituir   ANDERSON GONÇALVES ASSUNÇÃO (Siape: 2056808 ), Secretário da Procuradoria-Geral,
Código  FG-2,  em seu afastamento  por  motivo  de  La

#### Sentence tokenization
Having cleaned our documents we will split them into sentences. The **data_processing.split_text_sentences** function will return a DataFrame with: The sentences, a unique ID for each sentence and the index of the document each sentence is a part of.

In [9]:
df_sentences = data_processing.split_text_sentences(df_docs['text'])
pd.set_option("display.max_colwidth", 0)
print(df_sentences.shape)
df_sentences.head()

(2512, 3)


,document,sentence,sentence_id
0,25644,"Documento gerado sob autenticação Nº AHX.976.775.357, disponível no endereço http://www.ufrgs.br/autenticacao Documento certificado eletronicamente, conforme Portaria nº 3362/2016, que institui o Sistema de Documentos Eletrônicos da UFRGS.",0
1,25644,"1/1 PORTARIA Nº 5866 de 08/08/2016 O PRÓ-REITOR DE GESTÃO DE PESSOAS DA UNIVERSIDADE FEDERAL DO RIO GRANDE DO SUL, no uso de suas atribuições, considerando o disposto na Portaria nº 5469, de 04 de outubro de 2012 RESOLVE.",1
2,25644,"Conceder à servidora LISIANE RAMOS VILK, ocupante do cargo de Administrador de Edifícios - 701400, lotada na Faculdade de Arquitetura, SIAPE 2325261, o percentual de 25% (vinte e cinco por cento) de Incentivo à Qualificação, a contar de 15/07/2016, tendo em vista a conclusão do curso de Graduação em Administração - Bacharelado, conforme o Processo nº 23078.015333/2016-19.",2
3,25644,MAURÍCIO VIÉGAS DA SILVA Pró-Reitor de Gestão de Pessoas,3
4,25645,"Documento gerado sob autenticação Nº SDG.667.383.ILF, disponível no endereço http://www.ufrgs.br/autenticacao Documento certificado eletronicamente, conforme Portaria nº 3362/2016, que institui o Sistema de Documentos Eletrônicos da UFRGS.",4


#### Dealing with duplicate sentences
Now that we have our dataset of sentences we'll go to the last step which is identifying and categorizing duplicate sentences. Since we need to manually annotate the dataset for training we can group duplicate sentences to facilitate the anottation process. The document we have are fairly standardized so there are a few duplicates we will find.

Let's group sentences by their textual content. Each group will be composed of identical sentences.

In [10]:
test_group = df_sentences.groupby('sentence')

In [11]:
#pd.set_option("display.max_rows", 400)
#test_group.describe()

We loop each group of duplicated sentences and create a dictionary where the key will be the index of the first sentence of the group and the value is the index of all sentences equal to the first.

In [12]:
duplicates = {}
for i in test_group:
    dup_list = i[1]['sentence_id'].tolist()
    duplicates[dup_list[0]] = dup_list


We create a dataframe which will contain only unique sentences. The sentences that have identic pairs will have all of their indexes in a list in the 'duplicates' column. This will allow us to replicate the labels after the anotattion process.

In [13]:
df_unique_sentences = df_sentences.copy()
df_unique_sentences['duplicates'] = pd.Series(duplicates, index=df_sentences.index)
df_unique_sentences = df_unique_sentences.dropna(subset=['duplicates'])
df_unique_sentences['label'] = ""

We can see that the unique dataframe has less sentences which represen the duplicates we removed

In [14]:
pprint(df_sentences.shape)
pprint(df_unique_sentences.shape)

(2512, 3)
(1963, 5)


#### Manual Removal of Bad Sentences
Some sentences are not going to be helpful on our final train and test data. They either have just one or no entities we are interested in. Because of that we'll remove them from our dataset.

In [15]:
pprint(df_unique_sentences.shape)
df_unique_sentences = df_unique_sentences[df_unique_sentences['sentence'].str.len() > 20]
df_unique_sentences = df_unique_sentences[~df_unique_sentences['sentence'].str.startswith("Documento gerado sob")]
df_unique_sentences = df_unique_sentences[~df_unique_sentences['sentence'].str.startswith("Solicitação nº")]
df_unique_sentences = df_unique_sentences[~df_unique_sentences['sentence'].str.startswith("Processo")]
df_unique_sentences = df_unique_sentences[~df_unique_sentences['sentence'].str.contains("RESOLVE")]
pprint(df_unique_sentences.shape)

(1963, 5)
(729, 5)


In [16]:
#pprint(df_unique_sentences.to_json(lines=True, orient = 'records'))
df_docs.to_json(os.path.join(path_to_train, 'ufrgs_documents.jsonl'),lines=True, orient = 'records')
df_unique_sentences.to_json(os.path.join(path_to_train, 'unique_sentences.jsonl'),lines=True, orient = 'records')

#### Assistive Auto Labelling
Before we import our sentences into doccano for manual labelling, we'll use the auto_tagger to automatically label them. This will make it easier to do the manual labelling as a few entities will already be correctly or almost correctly identified.

In [17]:
# Load the auto_tagger model
from transformers import pipeline
assistive_auto_tagger = pipeline(
    "token-classification",
    model='auto_tagger',
    aggregation_strategy="average",
    ignore_labels=[""],
    device=0
)

In [26]:
# Run the sentences through our NER tagging model
df_unique_sentences['tagged_entities'] = df_unique_sentences['sentence'].apply(lambda x: assistive_auto_tagger(x[:512]))

c:\Users\arthu\anaconda3\envs\NLP22\lib\site-packages\transformers\pipelines\base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [27]:
df_unique_sentences['tagged_entities'].head(10)

2     [{'entity_group': 'O', 'score': 0.7762933, 'word': 'Conceder à servidora', 'start': 0, 'end': 20}, {'entity_group': 'PERSON', 'score': 0.6477044, 'word': 'LISIANE RAMOS VILK', 'start': 21, 'end': 39}, {'entity_group': 'O', 'score': 0.76407486, 'word': ', ocupante do cargo de Administrador de Edifícios - 701400, lotada na Faculdade de Arquitetura, SIAPE 2325261, o percentual de 25 % ( vinte e cinco por cento ) de Incentivo à Qualificação, a contar de 15 / 07 / 2016, tendo em vista a conclusão do curso de Graduação em Administração - Bacharelado, conforme o Processo nº 23078. 015333 / 2016 - 19.', 'start': 39, 'end': 374}]                                                                                          
3     [{'entity_group': 'PERSON', 'score': 0.6592996, 'word': 'MAURÍCIO VIÉGAS DA SILVA', 'start': 0, 'end': 24}, {'entity_group': 'O', 'score': 0.38087556, 'word': 'Pró - Reitor de Gestão de Pessoas', 'start': 25, 'end': 56}]                                                 

In [28]:
# Reformat the classified entities text to match the original sentence.
df_unique_sentences['tagged_entities'] = df_unique_sentences.apply(lambda x: data_processing.reformat_sentence(x['sentence'], x['tagged_entities']), axis=1)

In [29]:
df_unique_sentences['tagged_entities'].head(10)

2     [{'entity_group': 'O', 'score': 0.7762933, 'word': 'Conceder à servidora', 'start': 0, 'end': 20}, {'entity_group': 'PERSON', 'score': 0.6477044, 'word': 'LISIANE RAMOS VILK', 'start': 21, 'end': 39}, {'entity_group': 'O', 'score': 0.76407486, 'word': ', ocupante do cargo de Administrador de Edifícios - 701400, lotada na Faculdade de Arquitetura, SIAPE 2325261, o percentual de 25% (vinte e cinco por cento) de Incentivo à Qualificação, a contar de 15/07/2016, tendo em vista a conclusão do curso de Graduação em Administração - Bacharelado, conforme o Processo nº 23078.015333/2016-19.', 'start': 40, 'end': 375}]                                                                                         
3     [{'entity_group': 'PERSON', 'score': 0.6592996, 'word': 'MAURÍCIO VIÉGAS DA SILVA', 'start': 0, 'end': 24}, {'entity_group': 'O', 'score': 0.38087556, 'word': 'Pró-Reitor de Gestão de Pessoas', 'start': 25, 'end': 56}]                                                                

In [38]:
# Function that takes a Hugging Face NER Pipeline dictionary output and formats it in a format that can be used by Doccano for manual labelling.
# This is useful when using the NER Pipeline as an assistive auto labelling before doing the manual work.
def ner_entity_dict_to_doccano_jsonl(tagged_entities):
    # Join the entities texts to get the full sentence
    full_sentence_text = ""
    entities_metadata = []
    for entity in tagged_entities:
        full_sentence_text = full_sentence_text + entity['word'] + ' '
        if entity['entity_group'] != 'O':
            entities_metadata.append([entity['start'], entity['end'], entity['entity_group']])
    full_sentence_text = re.sub(' ,', ',', full_sentence_text)
    return full_sentence_text, entities_metadata

    # Get a list of the entities in the format [START, END, TYPE] for each of them.

In [36]:
df_unique_sentences.head()

,document,sentence,sentence_id,duplicates,label,tagged_entities
2,25644,"Conceder à servidora LISIANE RAMOS VILK, ocupante do cargo de Administrador de Edifícios - 701400, lotada na Faculdade de Arquitetura, SIAPE 2325261, o percentual de 25% (vinte e cinco por cento) de Incentivo à Qualificação, a contar de 15/07/2016, tendo em vista a conclusão do curso de Graduação em Administração - Bacharelado, conforme o Processo nº 23078.015333/2016-19.",2,[2],,"[{'entity_group': 'O', 'score': 0.7762933, 'word': 'Conceder à servidora', 'start': 0, 'end': 20}, {'entity_group': 'PERSON', 'score': 0.6477044, 'word': 'LISIANE RAMOS VILK', 'start': 21, 'end': 39}, {'entity_group': 'O', 'score': 0.76407486, 'word': ', ocupante do cargo de Administrador de Edifícios - 701400, lotada na Faculdade de Arquitetura, SIAPE 2325261, o percentual de 25% (vinte e cinco por cento) de Incentivo à Qualificação, a contar de 15/07/2016, tendo em vista a conclusão do curso de Graduação em Administração - Bacharelado, conforme o Processo nº 23078.015333/2016-19.', 'start': 40, 'end': 375}]"
3,25644,MAURÍCIO VIÉGAS DA SILVA Pró-Reitor de Gestão de Pessoas,3,"[3, 94, 156, 168, 173, 272, 276, 280, 284, 288, 292, 296, 300, 304, 324, 328, 332, 336, 345, 349, 353, 357, 416, 421, 426, 431, 436, 446, 451, 456, 461, 466, 471, 476, 481, 486, 491, 496, 501, 506, 511, 515, 519, 523, 527, 531, 535, 539, 543, 547, 553, 557, 561, 565, 569, 573, 578, 583, 588, 596, 602, 608, 632, 636, 640, 644, 648, 652, 656, 660, 664, 668, 672, 676, 680, 684, 688, 692, 696, 700, 704, 708, 712, 716, 720, 724, 728, 732, 736, 740, 744, 748, 752, 756, 760, 772, 783, 787, 791, 1040, ...]",,"[{'entity_group': 'PERSON', 'score': 0.6592996, 'word': 'MAURÍCIO VIÉGAS DA SILVA', 'start': 0, 'end': 24}, {'entity_group': 'O', 'score': 0.38087556, 'word': 'Pró-Reitor de Gestão de Pessoas', 'start': 25, 'end': 56}]"
6,25645,"Autorizar o afastamento do país de CRISTINE MARIA WARMLING, Professor do Magistério Superior, lotada e em exercício no Departamento de Odontologia Preventiva e Social da Faculdade de Odontologia, com a finalidade de participar do ""3ème Congrès de la Societé Internationale d'Ergologie"", em Aix-en-Provence - França, no período compreendido entre 28/08/2016 e 01/09/2016, com ônus limitado.",6,[6],,"[{'entity_group': 'O', 'score': 0.68199015, 'word': 'Autorizar o afastamento do país de', 'start': 0, 'end': 34}, {'entity_group': 'PERSON', 'score': 0.61941856, 'word': 'CRISTINE MARIA WARMLING', 'start': 35, 'end': 58}, {'entity_group': 'O', 'score': 0.658246, 'word': ', Professor do Magistério Superior, lotada e em exercício no Departamento de Odontologia Preventiva e Social da Faculdade de Odontologia, com a finalidade de participar do ""3ème Congrès de la Societé Internationale d'Ergologie"", em Aix-en-Provence - França, no período compreendido entre 28/08/2016 e 01/09/2016, com ônus limitado.', 'start': 59, 'end': 390}]"
8,25645,CARLOS ALEXANDRE NETTO Reitor,8,"[8, 13, 18, 23, 28, 38, 43, 63, 72, 81, 99, 104, 109, 114, 132, 137, 142, 147, 152, 161]",,"[{'entity_group': 'PERSON', 'score': 0.63235503, 'word': 'CARLOS ALEXANDRE NETTO', 'start': 0, 'end': 22}, {'entity_group': 'O', 'score': 0.418368, 'word': 'Reitor', 'start': 23, 'end': 29}]"
11,25646,"Autorizar o afastamento do país de ANDRE DIAS MORTARI, Assistente em Administração, lotado e em exercício na Escola de Administração, com a finalidade de participar do ""IV Congreso Internacional Red Pilares"", em Cartagena - Colombia, no período compreendido entre 29/08/2016 e 02/09/2016, com ônus limitado.",11,[11],,"[{'entity_group': 'O', 'score': 0.6808941, 'word': 'Autorizar o afastamento do país de', 'start': 0, 'end': 34}, {'entity_group': 'PERSON', 'score': 0.60456276, 'word': 'ANDRE DIAS MORTARI', 'start': 35, 'end': 53}, {'entity_group': 'O', 'score': 0.72398305, 'word': ', Assistente em Administração, lotado e em exercício na Escola de Administração, com a finalidade de participar do ""IV Congreso Internacional Red Pilares"", em 

In [42]:
df_unique_sentences['tagged_entities'].loc[2]
print(ner_entity_dict_to_doccano_jsonl(df_unique_sentences['tagged_entities'].loc[2]))
df_unique_sentences['label'] = df_unique_sentences.apply(lambda x: ner_entity_dict_to_doccano_jsonl(x['tagged_entities'])[1],axis=1)

('Conceder à servidora LISIANE RAMOS VILK, ocupante do cargo de Administrador de Edifícios - 701400, lotada na Faculdade de Arquitetura, SIAPE 2325261, o percentual de 25% (vinte e cinco por cento) de Incentivo à Qualificação, a contar de 15/07/2016, tendo em vista a conclusão do curso de Graduação em Administração - Bacharelado, conforme o Processo nº 23078.015333/2016-19. ', [[21, 39, 'PERSON']])


In [45]:
df_unique_sentences.to_json(os.path.join(path_to_train, 'unique_sentences.jsonl'),lines=True, orient = 'records')

In [46]:
df_unique_sentences[df_unique_sentences['sentence_id'] == 2507]

,document,sentence,sentence_id,duplicates,label,tagged_entities
2507,93672,2/2 SIAPE Nome do Servidor Classe PadrãoAnterior Novo Padrão Válida em 1908270 LISIANE BERNARDO DA SILVA E 05 06 28/06/2019 3002666 LIZIANE GONZALEZ D 01 02 12/06/2019 0359404 LUCIA DE FATIMA AMBOS C 15 16 02/06/2019 3001885 LUCIELE DOS SANTOS DA COSTA E 01 02 04/06/2019 1501710 MARCO ANTONIO MILLER D 09 10 27/06/2019 1055387 MARCOS LUIZ HINTERHOLZ E 03 04 02/06/2019 2185056 MAURA BOMBARDELLI E 03 04 30/06/2019 2319024 MICHELE UTPOTT E 02 03 13/06/2019 2317068 OTÁVIO LUÍS DA SILVA BARRADAS D 02 03 06/06/2019 2316407 PATRÍCIA ZIOMKOWSKI D 02 03 06/06/2019 1796567 RAFAEL ANDRIOLI RASCH D 06 07 25/06/2019 2187262 RAFAEL MARQUES MÜLLER D 03 04 22/06/2019 1395123 RICARDO ESTEVAM DE ANDRADE SANTOS D 01 02 27/06/2019 1423820 ROSE KELLY PIRES MARINHO D 01 02 27/06/2019 1697724 SAMUEL DOS SANTOS SALIMEN E 02 03 30/06/2019 3002061 SANDRA CRISTINA DE ARAUJO D 01 02 11/06/2019 3002675 THAÍS SPOHR CHRIST C 01 02 13/06/2019 1972210 TOBIAS DOS SANTOS GOMES D 01 02 27/06/2019 2324135 VALENCIA CRISTINA MEIER D 02 03 20/06/2019 2319305 VIANEI ELIZANDRO ZAMBONI D 02 03 22/06/2019 2321117 WAGNER MACHADO DA SILVA E 02 03 17/06/2019 1903536 ZULEIKA DE SOUZA BRANCO E 05 06 05/06/2019 MAURÍCIO VIÉGAS DA SILVA Pró-Reitor de Gestão de Pessoas,2507,[2507],"[[79, 104, PERSON], [131, 147, PERSON], [174, 195, PERSON], [222, 249, PERSON], [276, 296, PERSON], [323, 345, PERSON], [372, 389, PERSON], [416, 430, PERSON], [457, 486, PERSON]]","[{'entity_group': 'O', 'score': 0.73127943, 'word': '2/2 SIAPE Nome do Servidor Classe PadrãoAnterior Novo Padrão Válida em 1908270', 'start': 0, 'end': 78}, {'entity_group': 'PERSON', 'score': 0.60318124, 'word': 'LISIANE BERNARDO DA SILVA', 'start': 79, 'end': 104}, {'entity_group': 'O', 'score': 0.7948015, 'word': ' 05 06 28/06/2019 3002666', 'start': 105, 'end': 130}, {'entity_group': 'PERSON', 'score': 0.6116347, 'word': 'LIZIANE GONZALEZ', 'start': 131, 'end': 147}, {'entity_group': 'O', 'score': 0.79620516, 'word': ' 01 02 12/06/2019 0359404', 'start': 148, 'end': 173}, {'entity_group': 'PERSON', 'score': 0.57497036, 'word': 'LUCIA DE FATIMA AMBOS', 'start': 174, 'end': 195}, {'entity_group': 'O', 'score': 0.78663146, 'word': ' 15 16 02/06/2019 3001885', 'start': 196, 'end': 221}, {'entity_group': 'PERSON', 'score': 0.5348293, 'word': 'LUCIELE DOS SANTOS DA COSTA', 'start': 222, 'end': 249}, {'entity_group': 'O', 'score': 0.7869703, 'word': ' 01 02 04/06/2019 1501710', 'start': 250, 'end': 275}, {'entity_group': 'PERSON', 'score': 0.5773332, 'word': 'MARCO ANTONIO MILLER', 'start': 276, 'end': 296}, {'entity_group': 'O', 'score': 0.792752, 'word': ' 09 10 27/06/2019 1055387', 'start': 297, 'end': 322}, {'entity_group': 'PERSON', 'score': 0.5987951, 'word': 'MARCOS LUIZ HINTERHOLZ', 'start': 323, 'end': 345}, {'entity_group': 'O', 'score': 0.7839741, 'word': ' 03 04 02/06/2019 2185056', 'start': 346, 'end': 371}, {'entity_group': 'PERSON', 'score': 0.61466813, 'word': 'MAURA BOMBARDELLI', 'start': 372, 'end': 389}, {'entity_group': 'O', 'score': 0.78075325, 'word': ' 03 04 30/06/2019 2319024', 'start': 390, 'end': 415}, {'entity_group': 'PERSON', 'score': 0.49328786, 'word': 'MICHELE UTPOTT', 'start': 416, 'end': 430}, {'entity_group': 'O', 'score': 0.7886432, 'word': ' 02 03 13/06/2019 2317068', 'start': 431, 'end': 456}, {'entity_group': 'PERSON', 'score': 0.5957611, 'word': 'OTÁVIO LUÍS DA SILVA BARRADAS', 'start': 457, 'end': 486}, {'entity_group': 'O', 'score': 0.8131571, 'word': ' 02 03 06/06/201', 'start': 487, 'end': 503}]"
